INSTALL THE FOLLOWING PYTHON PACKAGES FIRST BEFORE RUNNING THE PROGRAM

1) Numpy
2) NNFS - for the Spiral dataset
3) scikit-learn - for the iris dataset

In [31]:
# Library imports
import numpy as np

Create classes for modularity

In [32]:
# Hidden Layers
# Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))


    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


In [33]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [34]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        # Gradient of BCE loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = y_pred.shape[0]

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # Use the first sample to count them
        labels = len(dvalues[0])

        # Check if labels are sparse, turn them into one-hot vector values
        # the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


<!-- Star -->

In [35]:
# Start of Optimizers

class Optimizer_SGD:
    # Initialie the optimizer - default learning rate to 1
    def __init__(self, learning_rate=1.0):
        self.learning_rate = learning_rate
        self.iterations = 0

    # Update the parameters
    def update_params(self, layer):
        layer.weights += -self.learning_rate * layer.dweights
        layer.biases += -self.learning_rate * layer.dbiases
    
    def post_update_params(self):
        self.iterations += 1

class Optimizer_Momentum:
    # Initialize the optimizer - default learning rate to 1 and momentum to 0.9
    def __init__(self, learning_rate=1.0, momentum=0.9):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.iterations = 0

    # Update the parameters
    def update_params(self, layer):
        # If layer does not contain momentum arrays, create them filled with zeros
        if not hasattr(layer, 'weight_momentums'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)

        # Build weight updates with momentum
        weight_updates = (self.momentum * layer.weight_momentums) - (self.learning_rate * layer.dweights)
        layer.weight_momentums = weight_updates

        # Build bias updates with momentum
        bias_updates = (self.momentum * layer.bias_momentums) - (self.learning_rate * layer.dbiases)
        layer.bias_momentums = bias_updates

        # Update weights and biases using updates
        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1

class Optimizer_Adagrad:
    # Initialize the optimizer - default learning rate to 1 and epsilon to 1e-7
    def __init__(self, learning_rate=1.0, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.iterations = 0

    # Update the parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays, create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update cache with squared current gradients
        layer.weight_cache += layer.dweights ** 2
        layer.bias_cache += layer.dbiases ** 2

        # Vanilla SGD parameter update + normalization with square rooted cache
        layer.weights += -self.learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

class Optimizer_LearningRateDecay:
    # Initialize the optimizer - default learning rate to 1 and decay to 0
    def __init__(self, learning_rate=1.0, decay=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))

    # Update the parameters
    def update_params(self, layer):
        layer.weights += -self.current_learning_rate * layer.dweights
        layer.biases += -self.current_learning_rate * layer.dbiases

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

Use most of the classes to create a functioning neural network, capable of performing a forward and backward pass

We can use a sample dataset from the Spiral module.  

We can also use the IRIS dataset.

In [36]:
# # Spiral Data
# import nnfs
# from nnfs.datasets import spiral_data

# # Create the dataset
# X, y = spiral_data(samples = 100, classes = 3)

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [37]:
# Iris Dataset
# From the scikit-learn library
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data # Features
y = iris.target # Target labels

print(X[:5])
print(X.shape)
print(y[:5])
print(y.shape)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
(150, 4)
[0 0 0 0 0]
(150,)


Forward Pass and Backward Pass

In [38]:
# Create the layers
dense1 = Layer_Dense(4, 8)  # 4 input features, 8 neurons in hidden layer
activation1 = Activation_ReLU()
dense2 = Layer_Dense(8, 3)  # 8 inputs from hidden layer, 3 output classes
activation2 = Activation_Softmax()

# Create loss function
loss_function = Loss_CategoricalCrossEntropy()

# Create optimizer (choose one: Optimizer_SGD, Optimizer_Momentum, Optimizer_Adagrad, Optimizer_LearningRateDecay)
optimizer = Optimizer_Adagrad(learning_rate=0.05)

# Training loop for 1000 epochs
for epoch in range(1000):
    # Forward Pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    
    # Calculate loss
    loss = loss_function.calculate(activation2.output, y)
    
    # Calculate accuracy
    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions == y)
    
    # Backward Pass (Backpropagation)
    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()
    
    # Print progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch + 1}/1000, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

# Final results
print('\nTraining Complete!')
print(f'Final Loss: {loss:.4f}')
print(f'Final Accuracy: {accuracy:.4f}')
print("Optimizer Used:" + optimizer.__class__.__name__)

Epoch 100/1000, Loss: 0.1777, Accuracy: 0.9800
Epoch 200/1000, Loss: 0.1062, Accuracy: 0.9800
Epoch 200/1000, Loss: 0.1062, Accuracy: 0.9800
Epoch 300/1000, Loss: 0.0835, Accuracy: 0.9733
Epoch 300/1000, Loss: 0.0835, Accuracy: 0.9733
Epoch 400/1000, Loss: 0.0727, Accuracy: 0.9800
Epoch 400/1000, Loss: 0.0727, Accuracy: 0.9800
Epoch 500/1000, Loss: 0.0664, Accuracy: 0.9800
Epoch 500/1000, Loss: 0.0664, Accuracy: 0.9800
Epoch 600/1000, Loss: 0.0623, Accuracy: 0.9800
Epoch 600/1000, Loss: 0.0623, Accuracy: 0.9800
Epoch 700/1000, Loss: 0.0593, Accuracy: 0.9800
Epoch 700/1000, Loss: 0.0593, Accuracy: 0.9800
Epoch 800/1000, Loss: 0.0571, Accuracy: 0.9800
Epoch 800/1000, Loss: 0.0571, Accuracy: 0.9800
Epoch 900/1000, Loss: 0.0554, Accuracy: 0.9800
Epoch 900/1000, Loss: 0.0554, Accuracy: 0.9800
Epoch 1000/1000, Loss: 0.0540, Accuracy: 0.9800

Training Complete!
Final Loss: 0.0540
Final Accuracy: 0.9800
Optimizer Used:Optimizer_Adagrad
Epoch 1000/1000, Loss: 0.0540, Accuracy: 0.9800

Training 

In [39]:
def train_with_optimizer(optimizer, optimizer_name):
    # Reset layers
    dense1 = Layer_Dense(4, 8)
    activation1 = Activation_ReLU()
    dense2 = Layer_Dense(8, 3)
    activation2 = Activation_Softmax()
    loss_function = Loss_CategoricalCrossEntropy()
    
    losses = []
    accuracies = []
    
    # Training loop
    for epoch in range(1000):
        # Forward Pass
        dense1.forward(X)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        activation2.forward(dense2.output)
        
        # Calculate loss and accuracy
        loss = loss_function.calculate(activation2.output, y)
        predictions = np.argmax(activation2.output, axis=1)
        accuracy = np.mean(predictions == y)
        
        losses.append(loss)
        accuracies.append(accuracy)
        
        # Backward Pass
        loss_function.backward(activation2.output, y)
        activation2.backward(loss_function.dinputs)
        dense2.backward(activation2.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)
        
        # Update weights
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)
        optimizer.post_update_params()
    
    return losses, accuracies

# Function to find stabilization epoch
def find_stabilization_epoch(losses, threshold=0.001, window=50):
    """
    Find the epoch where loss stabilizes.
    Stabilization = loss variance over a window falls below threshold
    """
    for i in range(window, len(losses)):
        loss_window = losses[i-window:i]
        variance = np.var(loss_window)
        if variance < threshold:
            return i - window + 1
    return len(losses)

# Train with AdaGrad
print("="*60)
print("TRAINING WITH ADAGRAD")
print("="*60)
adagrad_optimizer = Optimizer_Adagrad(learning_rate=0.05)
adagrad_losses, adagrad_accuracies = train_with_optimizer(adagrad_optimizer, "AdaGrad")

# Train with Momentum
print("="*60)
print("TRAINING WITH MOMENTUM")
print("="*60)
momentum_optimizer = Optimizer_Momentum(learning_rate=0.05, momentum=0.9)
momentum_losses, momentum_accuracies = train_with_optimizer(momentum_optimizer, "Momentum")

# Find stabilization epochs
adagrad_stabilization = find_stabilization_epoch(adagrad_losses)
momentum_stabilization = find_stabilization_epoch(momentum_losses)

# Analysis

print("\n1. LOSS STABILIZATION ANALYSIS")
print("-" * 60)
print(f"AdaGrad:")
print(f"  - Stabilization Epoch: {adagrad_stabilization}")
print(f"  - Loss at Stabilization: {adagrad_losses[adagrad_stabilization-1]:.6f}")
print(f"  - Final Loss (Epoch 1000): {adagrad_losses[-1]:.6f}")

print(f"\nMomentum:")
print(f"  - Stabilization Epoch: {momentum_stabilization}")
print(f"  - Loss at Stabilization: {momentum_losses[momentum_stabilization-1]:.6f}")
print(f"  - Final Loss (Epoch 1000): {momentum_losses[-1]:.6f}")

print(f"\nStabilization Difference:")
if adagrad_stabilization < momentum_stabilization:
    diff = momentum_stabilization - adagrad_stabilization
    print(f"  - AdaGrad stabilized {diff} epochs FASTER than Momentum")
else:
    diff = adagrad_stabilization - momentum_stabilization
    print(f"  - Momentum stabilized {diff} epochs FASTER than AdaGrad")

print("\n2. ACCURACY ANALYSIS")
print("-" * 60)
print(f"AdaGrad:")
print(f"  - Final Accuracy: {adagrad_accuracies[-1]:.6f} ({adagrad_accuracies[-1]*100:.2f}%)")
print(f"  - Accuracy at Stabilization: {adagrad_accuracies[adagrad_stabilization-1]:.6f}")

print(f"\nMomentum:")
print(f"  - Final Accuracy: {momentum_accuracies[-1]:.6f} ({momentum_accuracies[-1]*100:.2f}%)")
print(f"  - Accuracy at Stabilization: {momentum_accuracies[momentum_stabilization-1]:.6f}")


TRAINING WITH ADAGRAD
TRAINING WITH MOMENTUM
TRAINING WITH MOMENTUM

1. LOSS STABILIZATION ANALYSIS
------------------------------------------------------------
AdaGrad:
  - Stabilization Epoch: 108
  - Loss at Stabilization: 0.414939
  - Final Loss (Epoch 1000): 0.078077

Momentum:
  - Stabilization Epoch: 45
  - Loss at Stabilization: 0.201411
  - Final Loss (Epoch 1000): 0.046732

Stabilization Difference:
  - Momentum stabilized 63 epochs FASTER than AdaGrad

2. ACCURACY ANALYSIS
------------------------------------------------------------
AdaGrad:
  - Final Accuracy: 0.980000 (98.00%)
  - Accuracy at Stabilization: 0.940000

Momentum:
  - Final Accuracy: 0.980000 (98.00%)
  - Accuracy at Stabilization: 0.980000

1. LOSS STABILIZATION ANALYSIS
------------------------------------------------------------
AdaGrad:
  - Stabilization Epoch: 108
  - Loss at Stabilization: 0.414939
  - Final Loss (Epoch 1000): 0.078077

Momentum:
  - Stabilization Epoch: 45
  - Loss at Stabilization: 0.2